In [1]:
import matplotlib
%matplotlib inline

import numpy as np
import time
import cv2

import os
from lava.magma.core.run_configs import Loihi2SimCfg, Loihi2HwCfg
from lava.magma.core.run_conditions import RunSteps, RunContinuous
from lava.magma.core.decorator import implements, requires, tag
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.magma.core.model.py.ports import PyOutPort, PyInPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.process.ports.ports import OutPort, InPort
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.resources import CPU
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.proc.io.sink import RingBuffer

from lava.lib.peripherals.dvs.process import PropheseeCamera
from metavision_core.utils import get_sample
from metavision_sdk_cv import TrailFilterAlgorithm, ActivityNoiseFilterAlgorithm
from metavision_sdk_core import PolarityFilterAlgorithm

from multiprocessing import Pipe
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import IPython


from metavision_core.event_io import RawReader
import tonic
from tonic.transforms import Compose
from metavision_ml.preprocessing import histo, histo_quantized
from metavision_core.event_io import RawReader

from lava.proc.embedded_io.spike import PyToNxAdapter, NxToPyAdapter
from lava.proc.lif.process import LIF, LogConfig
from lava.proc.dense.process import Dense
from lava.proc.sparse.process import Sparse
import logging

from utils import VisProcess
from scipy.sparse import csr_matrix

/home/pweidel/prophesee_2_venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/metavision_core_ml/preprocessing/viz.py:41: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _viz_events(events, img):



## Download raw event recording

In [2]:
SEQUENCE_FILENAME_RAW = "80_balls.raw"
# if the file doesn't exist, it will be downloaded from Prophesee's public sample server
get_sample(SEQUENCE_FILENAME_RAW)

reader = RawReader(SEQUENCE_FILENAME_RAW)
height, width = reader.get_size()
del reader

...99%, 18 MB, 6792 KB/s, 2 seconds passed

## Basic usage

Initialize PropheseeCamera using the path to the recording, in order to use a live camera, replace the path with an empty string ("").

In [3]:

# Init Processes
frame_input = PropheseeCamera(device=SEQUENCE_FILENAME_RAW,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

recv = VisProcess(shape=frame_input.s_out.shape[2:])

# Connect
frame_input.s_out.connect(recv.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

frame_input.run(condition=run_cnd, run_cfg=run_cfg)
frame_input.stop()

Exception ignored in: <function AbstractProcess.__del__ at 0x7fb857018dc0>
Traceback (most recent call last):
  File "/home/pweidel/lava-nc/lava/src/lava/magma/core/process/process.py", line 221, in __del__
    self.stop()
  File "/home/pweidel/lava-nc/lava/src/lava/magma/core/process/process.py", line 412, in stop
    if self.runtime:
  File "/home/pweidel/lava-nc/lava/src/lava/magma/core/process/process.py", line 275, in runtime
    return self._runtime
AttributeError: 'VisProcess' object has no attribute '_runtime'


TypeError: __init__() missing 1 required positional argument: 'sensor_size'

## Apply filters

As you see, the falling items in the recording have a trail. In order to remove the trails, we can apply the TrailFilterAlgorithm from the metavision_sdk. See the metavision documentation for more filters and their usage.

In [4]:
filters = [TrailFilterAlgorithm(width=width, height=height, threshold=100000),
           ActivityNoiseFilterAlgorithm(width=width, height=height, threshold=1000),]


# Init Processes
frame_input = PropheseeCamera(device=SEQUENCE_FILENAME_RAW,
                              filters=filters,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

recv = VisProcess(shape=frame_input.s_out.shape)

# Connect
frame_input.s_out.connect(recv.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

frame_input.run(condition=run_cnd, run_cfg=run_cfg)
frame_input.stop()

## Apply transformations

We also provide the option to use tonic transformations on the data as for example downsampling. See the tonic documentation to see a complete list and usage.

In [5]:
transformations = tonic.transforms.Compose(
    [
        tonic.transforms.Downsample(spatial_factor=0.5),
    ]
)

# Init Processes
frame_input = PropheseeCamera(device=SEQUENCE_FILENAME_RAW,
                              transformations=transformations,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

recv = VisProcess(shape=frame_input.s_out.shape)

# Connect
frame_input.s_out.connect(recv.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

frame_input.run(condition=run_cnd, run_cfg=run_cfg)
frame_input.stop()

## Custom transformation and manual output shape

If you need to write your own transformation, the automatic shape determination would not work. In that case, you can specify the output shape manually.

In [6]:
def expand_x_dim(events):
    events['x'] += 500
    return events
    

transformations = tonic.transforms.Compose(
    [
        expand_x_dim,
    ]
)

# Init Processes
frame_input = PropheseeCamera(device=SEQUENCE_FILENAME_RAW,
                              transformations=transformations,
                              sensor_shape=(height, width),
                              num_output_time_bins=1,
                              out_shape=(1, 2, height, width+500))

recv = VisProcess(shape=frame_input.s_out.shape)

# Connect
frame_input.s_out.connect(recv.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

frame_input.run(condition=run_cnd, run_cfg=run_cfg)
frame_input.stop()
                              

